#**Data Mining Task**
the implementation of FP Growth Algorithm

##**First we start by reading our data and visualize it!**

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('/content/Horizontal_Format.xlsx')

In [3]:
df.head()

,TiD,items
0,1,"M,O,N,K,E,Y"
1,2,"D,O,N,K,E,Y"
2,3,"M,A,K,E"
3,4,"M,U,C,K,Y"
4,5,"C,O,O,K,I,E"


In [4]:
list_item = df['items'].copy()
print(list_item)

0    M,O,N,K,E,Y
1    D,O,N,K,E,Y
2        M,A,K,E
3      M,U,C,K,Y
4    C,O,O,K,I,E
Name: items, dtype: object


##**Add all our transactions in a set to filter the repeated the items**

In [5]:
all_Transactions = []
for i in list_item:
  items_in_transaction = []
  temp_set = set()
  for j in i:
    if j != ',' and j not in temp_set:
      items_in_transaction.append(j)
      temp_set.add(j)  #AYA :to filter items that are repeated in the same transaction (ex: C O O K I E ---> C O K I E)
  all_Transactions.append(items_in_transaction)
print(all_Transactions)

[['M', 'O', 'N', 'K', 'E', 'Y'], ['D', 'O', 'N', 'K', 'E', 'Y'], ['M', 'A', 'K', 'E'], ['M', 'U', 'C', 'K', 'Y'], ['C', 'O', 'K', 'I', 'E']]


In [6]:
all_items = [] #Aya sherif
for i in list_item:
  for j in i:
    if j != ',':
      all_items.append(j)
print("All items:")
print(all_items)
unique_items = set(all_items)
print("Unique items:")
print(unique_items)

All items:
['M', 'O', 'N', 'K', 'E', 'Y', 'D', 'O', 'N', 'K', 'E', 'Y', 'M', 'A', 'K', 'E', 'M', 'U', 'C', 'K', 'Y', 'C', 'O', 'O', 'K', 'I', 'E']
Unique items:
{'A', 'N', 'D', 'M', 'Y', 'C', 'I', 'U', 'E', 'O', 'K'}


##**count the freq of each item and put the items in dictonary as keys and put the freq of each beside it**

In [7]:
dict_of_items = {}
for i in unique_items: # AYA : counts the frequency of each item
  count = 0
  for j in list_item:
    if i in j:
      count += 1
  dict_of_items[i] = count
print("Items and their frequencies:")
print(dict_of_items)

Items and their frequencies:
{'A': 1, 'N': 2, 'D': 1, 'M': 3, 'Y': 3, 'C': 2, 'I': 1, 'U': 1, 'E': 4, 'O': 3, 'K': 5}


##**Add some helper functions**

In [8]:
min_support = input("Enter the minimum support: ")
def isFrequent(count, min_support): #AYA : returns true if the itemset is frequent
  return count >= int(min_support)

frequentItemsets = {} #AYA :dictionary of frequent itemsets only
for i in dict_of_items:
  if isFrequent(dict_of_items[i], min_support):
    frequentItemsets[i] = dict_of_items[i]

print("Frequent itemsets:")
print(frequentItemsets)


Enter the minimum support: 3
Frequent itemsets:
{'M': 3, 'Y': 3, 'E': 4, 'O': 3, 'K': 5}


In [9]:
#descending function JANA
def descending(frequentItemsets):
  sorted_items=sorted(frequentItemsets.items(),key=lambda x:x[1],reverse=True) # JANA : reverse true means descending here, x[1] is the value aka count
  return sorted_items

sorted_items= dict(descending(frequentItemsets))
print("Frequent itemsets after sorting (descending):")
print(sorted_items)

Frequent itemsets after sorting (descending):
{'K': 5, 'E': 4, 'M': 3, 'Y': 3, 'O': 3}


In [10]:
print("Transactions before: ")
for transaction in all_Transactions:
  print(transaction)

Transactions before: 
['M', 'O', 'N', 'K', 'E', 'Y']
['D', 'O', 'N', 'K', 'E', 'Y']
['M', 'A', 'K', 'E']
['M', 'U', 'C', 'K', 'Y']
['C', 'O', 'K', 'I', 'E']


In [11]:
def remove_arrange(transaction, sorted_items):
 filtered = [i for i in transaction if i in frequentItemsets]
 arranged = sorted(filtered, key=lambda x: sorted_items[x], reverse=True)
 return arranged

print("Transactions after having their items filtered and arranged: ")
for transaction in all_Transactions:
   transaction =remove_arrange(transaction, sorted_items)
   print(transaction)

Transactions after having their items filtered and arranged: 
['K', 'E', 'M', 'O', 'Y']
['K', 'E', 'O', 'Y']
['K', 'E', 'M']
['K', 'M', 'Y']
['K', 'E', 'O']


##**Node class for each node in our tree**


In [12]:
#omar note : nodes for tree
class node:
    def __init__(self,item,count,parent):
        self.item=item
        self.count=count
        self.parent=parent
        self.child={}
        self.next_similar = None
    def Update_Frequency(self,count):
        self.count=count

##**Tree class**

In [13]:
#omar note : tree class using the nodes
class Tree:
  #omar note  : the init dont take any arguments that intialize needs in our class
    def __init__(self):
        self.root=node(None,None,None) # root ->> NULL
        self.header_table = {}
    def insert(self,Transaction):#list of sorted items --> Amr HAny
        current_node=self.root
        for i in Transaction:
            if i in current_node.child: # AMR :Update the Frequency
              current_node.child[i].Update_Frequency(current_node.child[i].count + 1)
            else: #AMR : create a newNode
              NewNode=node(i,1,current_node)
              current_node.child[i]=NewNode
              if i in self.header_table:
                self.update_header_table(i, NewNode)
              else:
                self.header_table[i] = NewNode
            current_node=current_node.child[i]
    def update_header_table(self,item,new_node):
      """Omar note link nodes with the same item from header table"""
      current_node_in_chain = self.header_table[item]
      while current_node_in_chain.next_similar is not None:
            current_node_in_chain = current_node_in_chain.next_similar
      current_node_in_chain.next_similar = new_node
    def get_paths(self,item):
       """Omar note : Get all paths ending with the specified item"""
       paths = []
       current_linked_node = self.header_table.get(item)
       while current_linked_node is not None:
        path = []
        temp_node = current_linked_node.parent

        while temp_node.item is not None:
          path.append(temp_node.item)
          temp_node = temp_node.parent

        if path:
          paths.append((path[::-1], current_linked_node.count))
        current_linked_node = current_linked_node.next_similar
       return paths
    def print_tree(self,node=None,indent=""):
       if node is None:
          node = self.root
          print("FP-Tree Structure:")
          print("Root")

       for item, child in node.child.items():
          print(f"{indent}├── {item}:{child.count}")
          self.print_tree(child, indent + "│   ")
    #omar note : delete node using the FP Algo
    def delete(self,item):
      nodes_to_visit = [self.root]
      while nodes_to_visit:
        current_node = nodes_to_visit.pop()
        for child_item_name in list(current_node.child.keys()):
          child_node = current_node.child[child_item_name]
          if child_item_name == item and len(child_node.child)==0:
            del current_node.child[child_item_name]
          else:
            nodes_to_visit.append(child_node)

In [14]:
tree = Tree()
for transaction in all_Transactions:
    arranged_trans = remove_arrange(transaction, sorted_items)
    if arranged_trans:
        tree.insert(arranged_trans)

In [15]:
tree.print_tree()

FP-Tree Structure:
Root
├── K:5
│   ├── E:4
│   │   ├── M:2
│   │   │   ├── O:1
│   │   │   │   ├── Y:1
│   │   ├── O:2
│   │   │   ├── Y:1
│   ├── M:1
│   │   ├── Y:1


##**Then we will do the FP Growth algorithm using Recursive algorithm**

In [16]:
def get_all_nodes(tree,item):
  """omar note: Get all nodes with the specified item from header table"""
  nodes_of_tree = []
  node = tree.header_table.get(item)
  while node is not None:
    nodes_of_tree.append(node)
    node = node.next_similar
  return nodes_of_tree

In [17]:
def fp_growth_algo(tree,prefix,min_support,Freq_patterns):
  """
  omar note :
  this is a fucntion that takes the tree as a parameter and min support
  where min support is the min support that counts a certain itemset as
  a frequent one and add to Freq patterns
  """
  #sort items based on there count
  items = sorted(tree.header_table.keys(), key=lambda x: sum(node.count for node in get_all_nodes(tree,x)))
  for item in items:
    new_pattern = prefix+[item]
    support = sum(node.count for node in get_all_nodes(tree,item))
    #omar note : check if their support are higher than the min if tho add to the array
    if support >= int(min_support):
      pattern_key = tuple(sorted(new_pattern))
      if pattern_key not in Freq_patterns:
        Freq_patterns[pattern_key] = support
        print(f"Pattern:{new_pattern}->Support:{support}")

      paths = tree.get_paths(item)
      if paths:
        conditional_tree = Tree()
        for path_data in paths:
          current_path = path_data[0]
          current_count = path_data[1]
          for _ in range(current_count):
            if current_path:
              conditional_tree.insert(current_path)

        if conditional_tree.header_table:
          fp_growth_algo(conditional_tree, new_pattern, min_support, Freq_patterns)

##**Test the FP Algo**

In [18]:
frequent_patterns = {}
for item, count in frequentItemsets.items():
    frequent_patterns[(item,)] = count
fp_growth_algo(tree, [], min_support, frequent_patterns)
print("All Frequent Patterns:")
sorted_patterns = sorted(frequent_patterns.items(), key=lambda x: (-len(x[0]), -x[1]))
for pattern, support in sorted_patterns:
    print(f"{list(pattern)} -> Support: {support}")

Pattern:['M', 'K']->Support:3
Pattern:['O', 'K']->Support:3
Pattern:['O', 'E']->Support:3
Pattern:['O', 'E', 'K']->Support:3
Pattern:['Y', 'K']->Support:3
Pattern:['E', 'K']->Support:4
All Frequent Patterns:
['E', 'K', 'O'] -> Support: 3
['E', 'K'] -> Support: 4
['K', 'M'] -> Support: 3
['K', 'O'] -> Support: 3
['E', 'O'] -> Support: 3
['K', 'Y'] -> Support: 3
['K'] -> Support: 5
['E'] -> Support: 4
['M'] -> Support: 3
['Y'] -> Support: 3
['O'] -> Support: 3


##**All possible frequent itemsets for the provided min support**

In [19]:
print("All frequent itemsets:")
sorted_patterns_L = sorted(frequent_patterns.items(), key=lambda x: (-len(x[0]), -x[1]), reverse = True)
current_length = 0
for pattern, support in sorted_patterns_L:
    pattern_length = len(pattern)
    if pattern_length != current_length:
        current_length = pattern_length
        print(f"\nL{current_length}:")
    print(f"  {list(pattern)} -> Support: {support}")

All frequent itemsets:

L1:
  ['M'] -> Support: 3
  ['Y'] -> Support: 3
  ['O'] -> Support: 3
  ['E'] -> Support: 4
  ['K'] -> Support: 5

L2:
  ['K', 'M'] -> Support: 3
  ['K', 'O'] -> Support: 3
  ['E', 'O'] -> Support: 3
  ['K', 'Y'] -> Support: 3
  ['E', 'K'] -> Support: 4

L3:
  ['E', 'K', 'O'] -> Support: 3


##**Get Association Rules**




In [20]:
#jana
from itertools import combinations #JANA : make combinations for every itemset that have more than 2 items

def generate_association_rules(frequent_patterns):
  generated_rules=[]

  for itemset in frequent_patterns.keys():
    if len(itemset) < 2:
      continue

    for i in range(1,len(itemset)):
      for LHS in combinations(itemset,i):
        RHS = tuple(sorted(set(itemset) - set(LHS)))
        generated_rules.append({
            "LHS":sorted(LHS),
            "RHS":sorted(RHS)
            })
  return generated_rules

In [21]:
generated_rules = generate_association_rules(frequent_patterns)

for rule in generated_rules:
    print(f"Association Rule: {rule['LHS']} -> {rule['RHS']}")

Association Rule: ['K'] -> ['M']
Association Rule: ['M'] -> ['K']
Association Rule: ['K'] -> ['O']
Association Rule: ['O'] -> ['K']
Association Rule: ['E'] -> ['O']
Association Rule: ['O'] -> ['E']
Association Rule: ['E'] -> ['K', 'O']
Association Rule: ['K'] -> ['E', 'O']
Association Rule: ['O'] -> ['E', 'K']
Association Rule: ['E', 'K'] -> ['O']
Association Rule: ['E', 'O'] -> ['K']
Association Rule: ['K', 'O'] -> ['E']
Association Rule: ['K'] -> ['Y']
Association Rule: ['Y'] -> ['K']
Association Rule: ['E'] -> ['K']
Association Rule: ['K'] -> ['E']



2.   evaluate the rules



In [22]:
def evaluate_rules(frequent_patterns,generated_rules, all_Transactions):
  #JANA : FUNC that print the confidince and ther lift values
  results=[]

  for rule in generated_rules:
    lhs=tuple(sorted(rule["LHS"]))
    rhs=tuple(sorted(rule["RHS"]))
    union = tuple(sorted(set(lhs) | set(rhs)))

    lhs_support = frequent_patterns[lhs]
    rhs_support = frequent_patterns[rhs]
    union_support = frequent_patterns[union]

    if(lhs_support == 0 or rhs_support == 0 or union_support == 0):
      continue

    confidence = union_support / lhs_support
    lift = confidence / (rhs_support / len(all_Transactions))

    results.append({
        "LHS":rule["LHS"],
        "RHS":rule["RHS"],
        "Confidence":confidence,
        "Lift":lift
    })

  return results



In [23]:
rules = evaluate_rules(frequent_patterns,generated_rules, all_Transactions)
for rule in rules:
    print(f"Association Rule: {rule['LHS']} -> {rule['RHS']}")
    print(f"Confidence: {rule['Confidence']}")
    print(f"Lift: {rule['Lift']}")


Association Rule: ['K'] -> ['M']
Confidence: 0.6
Lift: 1.0
Association Rule: ['M'] -> ['K']
Confidence: 1.0
Lift: 1.0
Association Rule: ['K'] -> ['O']
Confidence: 0.6
Lift: 1.0
Association Rule: ['O'] -> ['K']
Confidence: 1.0
Lift: 1.0
Association Rule: ['E'] -> ['O']
Confidence: 0.75
Lift: 1.25
Association Rule: ['O'] -> ['E']
Confidence: 1.0
Lift: 1.25
Association Rule: ['E'] -> ['K', 'O']
Confidence: 0.75
Lift: 1.25
Association Rule: ['K'] -> ['E', 'O']
Confidence: 0.6
Lift: 1.0
Association Rule: ['O'] -> ['E', 'K']
Confidence: 1.0
Lift: 1.25
Association Rule: ['E', 'K'] -> ['O']
Confidence: 0.75
Lift: 1.25
Association Rule: ['E', 'O'] -> ['K']
Confidence: 1.0
Lift: 1.0
Association Rule: ['K', 'O'] -> ['E']
Confidence: 1.0
Lift: 1.25
Association Rule: ['K'] -> ['Y']
Confidence: 0.6
Lift: 1.0
Association Rule: ['Y'] -> ['K']
Confidence: 1.0
Lift: 1.0
Association Rule: ['E'] -> ['K']
Confidence: 1.0
Lift: 1.0
Association Rule: ['K'] -> ['E']
Confidence: 0.8
Lift: 1.0


In [24]:
def check_strong_rules(rules, min_confidence): #JANA function to get which are strong and which not
    strong_rules = []
    for rule in rules:
        if rule['Confidence'] >= min_confidence:
            strong_rules.append(rule)
    return strong_rules

In [25]:
min_confidence = float(input("Enter the minimum confidence: "))
strong_rules = check_strong_rules(rules, min_confidence)
print("Strong Association Rules:")
for rule in strong_rules:
    print(f"Association Rule: {rule['LHS']} -> {rule['RHS']}")
    print(f"Confidence: {rule['Confidence']}")


Enter the minimum confidence: .8
Strong Association Rules:
Association Rule: ['M'] -> ['K']
Confidence: 1.0
Association Rule: ['O'] -> ['K']
Confidence: 1.0
Association Rule: ['O'] -> ['E']
Confidence: 1.0
Association Rule: ['O'] -> ['E', 'K']
Confidence: 1.0
Association Rule: ['E', 'O'] -> ['K']
Confidence: 1.0
Association Rule: ['K', 'O'] -> ['E']
Confidence: 1.0
Association Rule: ['Y'] -> ['K']
Confidence: 1.0
Association Rule: ['E'] -> ['K']
Confidence: 1.0
Association Rule: ['K'] -> ['E']
Confidence: 0.8
